In [2]:
import tensorflow as tf
import cv2
import numpy as np
from PIL import Image
from flask import request, jsonify, Flask
from keras.models import Sequential, load_model
from keras import backend as k
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import requests
from obj import images,classes

In [3]:
# Create Flask app
app = Flask(__name__)

In [4]:
# Function to load the models
def load_model_fn(path):
    model = load_model(path)
    print('Model loaded!')
    return model

load = load_model_fn('./model.h5') 

c:\Users\MG\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
c:\Users\MG\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\optimizers\base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model loaded!


In [5]:
# Function to preprocess the image
def preprocess_image(image_path, target_size):
    try:
        # Load the image
        image = Image.open(requests.get(image_path, stream=True).raw)
        # Convert the image to RGB
        rgb_image = image.convert("RGB")

        # Resize the RGB image
        resized_image = rgb_image.resize(target_size)

        # Convert the RGB image to a NumPy array
        image_array = np.array(resized_image)

        # Normalize the image by dividing pixel values by 255
        mean_value = 0.5
        std_value = 0.5
        image_array = (image_array - mean_value) / std_value

        # Add batch dimension
        image_array = np.expand_dims(image_array, axis=0)

        return image_array
    except Exception as e:
        print(f"Error preprocessing image: {str(e)}")
        return None

In [6]:
from flask_cors import CORS
# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all origins

In [7]:
@app.route('/predictApi', methods=['POST'])
def api():
    try:
        # Get the JSON data from the request
        data = request.json

        # Extract the image URL from the JSON data
        imageUrl = data.get('imageUrl')

        if not imageUrl:
            return jsonify({"error": "Image URL not provided."})

        # Preprocess the image using the provided URL
        image_arr = preprocess_image(imageUrl, target_size=(224, 224))

        # Handle error if image preprocessing fails
        if image_arr is None:
            return jsonify({"error": "Error processing image."})

        # Make predictions using the loaded model
        prediction = load.predict(image_arr)

        # Get the predicted class label index and confidence
        index_largest_value = np.argmax(prediction[0])
        confidence = prediction[0][index_largest_value]

        # Check if the confidence is below the threshold
        threshold = 0.7  # Set your desired confidence threshold here
        if confidence < threshold:
            return jsonify({"error": "Confidence below threshold."})

        result = classes[index_largest_value]

        # Check if the predicted class label is in the classes list
        if result not in classes:
            return jsonify({"error": "Photo not found."})

        # Return the prediction and confidence as JSON response
        return jsonify({"prediction": result, "confidence": float(confidence), "image": images[result]})
    except Exception as e:
        # Return error message as JSON response
        return jsonify({"error": str(e)})    

In [9]:
if __name__ == "__main__":
    # Run the Flask app
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
